### Check how many layers are frozen in :

https://github.com/ShehabMMohamed/TinyImageNet-KaggleCompetition

In [ ]:
from keras.applications import Xception
from keras.layers import Input, Dense

In [ ]:
img_height = 299
img_width = 299

input_tensor = Input(shape=(img_width, img_height, 3))

pre_trained_model = Xception(weights='imagenet', input_tensor=input_tensor, include_top=False, pooling='avg')
# Downloads 83.7Mb of model..., 

In [ ]:
pre_trained_model.layers[20].name
# 'block3_sepconv2' ( next are 'block3_sepconv2_bn' and 'conv2d_2' )

In [ ]:
pre_trained_model.summary()
#Total params: 20,861,480
#Trainable params: 20,806,952
#Non-trainable params: 54,528
# Input ... block3_sepconv2 ... block14_sepconv2 ... global_average_pooling2d_1

In [ ]:
### That's a lot of fine tuning...

### Position of block3_sepconv2 in xception : 

https://github.com/keras-team/keras-applications/blob/master/keras_applications/xception.py#L188

### Should check the layout of the PyTorch model too...

https://github.com/Cadene/pretrained-models.pytorch/blob/master/pretrainedmodels/models/xception.py#L137

Seems to be only 12 blocks, which is a little odd...

### See UMAP results on basic 'digits' raw image files (per documentation)

In [ ]:
! pip install umap-learn

In [ ]:
import numpy as np
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

In [ ]:
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

In [ ]:
digits = load_digits()
#print(digits.DESCR) # Documentation
#digits.images.shape # (1797, 8, 8)
digits.data.shape   # (1797, 64)

In [ ]:
import umap

In [ ]:
reducer = umap.UMAP(random_state=42)
reducer.fit(digits.data)  # <3

In [ ]:
embedding = reducer.transform(digits.data)
# Verify that the result of calling transform is
# idenitical to accessing the embedding_ attribute
assert(np.all(embedding == reducer.embedding_))
embedding.shape

In [ ]:
plt.scatter(embedding[:, 0], embedding[:, 1], c=digits.target, cmap='Spectral', s=5)
plt.gca().set_aspect('equal', 'datalim')
plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10))
plt.title('UMAP projection of the Digits dataset', fontsize=24);

### See UMAP results on plain MNIST raw image files

*  https://github.com/snakers4/playing_with_vae

In [ ]:
# ! pip install Pillow-SIMD  # Hmm : Missing jpeg library...

In [ ]:
import numpy as np

import torch
import torchvision.datasets

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1307,), (0.3081,)),
])

mnist_data = torchvision.datasets.MNIST('./data/mnist', download=True, 
  train=True, transform=transform, )
mnist_data_test = torchvision.datasets.MNIST('./data/mnist', download=True, 
  train=False, transform=transform, )

data_set = mnist_data

In [ ]:
batch_size=100

data_loader = torch.utils.data.DataLoader(
  data_set, batch_size=batch_size, #shuffle=True,
  num_workers=0, 
)

In [ ]:
proj_dim = 50

proj = torch.nn.Linear( 28*28, proj_dim )  # Does the initialisation 'better' than torch.randn()

In [ ]:
# This maps the result of the projection directly into the results numpy arrays
xs, ys = np.zeros( (len(data_set), proj_dim) ), np.zeros( (len(data_set),) )
for i_batch, (x_batch, y_batch) in enumerate(data_loader):
    x_proj = proj( x_batch.view(-1, 28*28) )
    #print(x_proj.size()); break
    xs[i_batch*batch_size:(i_batch+1)*batch_size, :] = x_proj.detach()
    ys[i_batch*batch_size:(i_batch+1)*batch_size] = y_batch.detach()    

In [ ]:
#i_batch*batch_size
base=59990; ys[base+0:base+10]

In [ ]:
import time
import umap

In [ ]:
t0 = time.time()
reducer = umap.UMAP(random_state=42)
reducer.fit(xs) 
print("Fitting %d-D took %d secs" % (proj_dim, time.time()-t0,) )  
# 400d : 74 secs
# 100d : 76 secs 
#  50d : 69 secs
#  35d : 68 secs
#  20d : 68 secs

In [ ]:
embedding = reducer.transform(xs)
embedding.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(embedding[:, 0], embedding[:, 1], c=ys, cmap='Spectral', s=5)
plt.gca().set_aspect('equal', 'datalim')
plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10))
plt.title('UMAP %d-D projection of MNIST' % (proj_dim,), fontsize=24);

###  Train CNN on dataset (with labels)

In [ ]:
# Ok, so now let's build a CNN to classify regular MNIST 
#   with last layer being 50D, and re-run the UMAP on that result...

In [ ]:
class BasicNet(torch.nn.Module):
    def __init__(self):
        super(BasicNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
        self.conv1_mp = torch.nn.MaxPool2d(2)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_mp = torch.nn.MaxPool2d(2)
        self.conv2_drop = torch.nn.Dropout2d()
        self.fc1 = torch.nn.Linear(320, 50)
        self.drop = torch.nn.Dropout()
        self.fc2 = torch.nn.Linear(50, 10)
        self.logsoftmax = torch.nn.LogSoftmax(dim=1)
        self.relu = torch.nn.ReLU()

    def forward(self, x, middle_layer=False):
        x = self.relu(self.conv1_mp(self.conv1(x)))
        x = self.relu(self.conv2_mp(self.conv2_drop(self.conv2(x))))
        x = x.view(-1, 320) # Flatten
        x = self.fc1(x)
        if middle_layer:
            return x
        x = self.drop(self.relu(x))
        x = self.fc2(x)
        return self.logsoftmax(x)

In [ ]:
model = BasicNet().to(device)

In [ ]:
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
batch_size=40

train_loader = torch.utils.data.DataLoader(
  data_set, batch_size=batch_size, #shuffle=True,
  num_workers=0, 
)

In [ ]:
model.train()
loss_fn = torch.nn.NLLLoss()

for epoch in range(5):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % (10000//batch_size) == 0:
            print('Train Epoch: {} [{:5d}/{:5d} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
# Just use the test dataset to evaluate our model
test_loader = torch.utils.data.DataLoader(
  mnist_data_test, batch_size=batch_size, #shuffle=True,
  num_workers=0, 
)

In [ ]:
model.eval()
test_loss, correct = 0., 0
for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)
    test_loss += loss_fn(output, target).item() # sum up batch loss
    pred = output.data.max(1, keepdim=True)[1] # get the indices of the max log-probability
    correct += pred.eq(target.data.view_as(pred)).cpu().sum().numpy()

test_loss /= len(test_loader.dataset)
print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    correct * 100. / len(test_loader.dataset)))

### Do UMAP on the final hidden layer

In [ ]:
# This maps the result of the projection directly into the results numpy arrays
xs_cnn, ys_cnn = np.zeros( (len(data_set), proj_dim) ), np.zeros( (len(data_set),) )
model.eval()
for i_batch, (x_batch, y_batch) in enumerate(data_loader):
    data = x_batch.to(device)
    output = model(data, middle_layer=True)
    x_cnn = output.cpu().detach().numpy()
    xs_cnn[i_batch*batch_size:(i_batch+1)*batch_size, :] = x_cnn
    ys_cnn[i_batch*batch_size:(i_batch+1)*batch_size] = y_batch.detach()    

In [ ]:
t0 = time.time()
reducer_cnn = umap.UMAP(random_state=42)
reducer_cnn.fit(xs_cnn) 
print("Fitting %d-D to CNN middle layer took %d secs" % (proj_dim, time.time()-t0,) )  
# 50d : 70 secs 

In [ ]:
embedding_cnn = reducer_cnn.transform(xs_cnn)
embedding_cnn.shape

In [ ]:
plt.figure(figsize=(8,8))
# Other potential colormaps : Spectral, tab10, tab20, tab20b?, terrain, jet
plt.scatter(embedding_cnn[:, 0], embedding_cnn[:, 1], c=ys_cnn, cmap='Spectral', s=5) 
plt.gca().set_aspect('equal', 'datalim')
plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10))
plt.title('UMAP %d-D CNN middle layer for MNIST' % (proj_dim,), fontsize=24);

### Ideas

*  Investigate what the UMAP 'reducer' and 'embeddings' are doing
   +   What does the embedding represent? 
   +   Is the embedding usable in its own right?
*  Train MNIST but leave several digits out
   +   Look at where they would lie in UMAP space
